In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs
from pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun
from pyphoplacecellanalysis.General.Batch.runBatch import run_diba_batch

from scripts.run_BatchAnalysis import post_compute_validate, _perform_plots, _on_complete_success_execution_session

active_global_batch_result_filename='global_batch_result_2023-05-30.pkl'
debug_print=True
enable_neptune = True

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper import neptune_output_figures
    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

    import os
    os.environ["NEPTUNE_API_TOKEN"] = neptune_kwargs['api_token']
    os.environ["NEPTUNE_PROJECT"] = neptune_kwargs['project']


Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
""" 
from pyphoplacecellanalysis.General.Batch.runBatch import main, BatchRun, run_diba_batch, run_specific_batch

"""
global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"


## NEPTUNE:
if enable_neptune:
	project = neptune.init_project(**neptune_kwargs)
	project["general/global_batch_result_filename"] = active_global_batch_result_filename
	project["general/global_data_root_parent_path"] = global_data_root_parent_path.as_posix()

## TODO: load the batch result initially:

## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

if enable_neptune:
	run = neptune.init_run() # rember to run.stop()
	run['parameters/global_batch_result_file_path'] = global_batch_result_file_path.as_posix()
	# project["general/data_analysis"].upload("data_analysis.ipynb")
	run["dataset/latest"].track_files(f"file://{global_batch_result_file_path}") # "s3://datasets/images"

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)

if enable_neptune:
	run["dataset/global_batch_run_progress_df"].upload(File.as_html(batch_progress_df)) # "path/to/test_preds.csv"
	run["dataset/global_batch_run_good_only_df"].upload(File.as_html(good_only_batch_progress_df)) # "path/to/test_preds.csv"
	
batch_progress_df

https://app.neptune.ai/commander.pho/PhoDibaLongShort2023/


c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/commander.pho/PhoDibaLongShort2023/e/PHOD-465
finalized_loaded_global_batch_result_pickle_path: W:\Data\global_batch_result_2023-05-30.pkl
Loading loaded session pickle file results : W:\Data\global_batch_result_2023-05-30.pkl... encountered exception 'BatchRun' object is not iterable while printing. Turning into a warning and continuing.
done.
no difference between provided and internal paths.


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,W:\Data\KDIBA\gor01\one\2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,W:\Data\KDIBA\pin01\one\fet11-04_21-20-3,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,W:\Data\KDIBA\pin01\one\redundant,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,W:\Data\KDIBA\pin01\one\showclus,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,W:\Data\KDIBA\pin01\one\sleep,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False


# Build `global_batch_run` pre-loading results (before execution)

In [3]:

# global_batch_result = loadData('global_batch_result.pkl')
global_batch_run = run_diba_batch(global_data_root_parent_path, execute_all=False, extant_batch_run=global_batch_run, debug_print=False)
# print(f'global_batch_result: {global_batch_run}')
global_batch_run

resusing extant_batch_run: BatchRun(global_data_root_parent_path=WindowsPath('W:/Data'), session_batch_status={IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_3-23-37')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6

BatchRun(global_data_root_parent_path=WindowsPath('W:/Data'), session_batch_status={IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_3-23-37')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-08_15-46-47')>: <SessionBa

# Run Batch Executions/Computations

In [4]:
## I got it doing the bare-minimum loading and computations, so it should be ready to update the laps and constraint the placefields to those. Then we should be able to set up the replays at the same time.
# finally, we then finish by computing.

## Execute with the custom arguments.
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=True, skip_extended_batch_computations=True, post_run_callback_fn=_on_complete_success_execution_session, **{'computation_functions_name_whitelist': active_computation_functions_name_whitelist, 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

https://app.neptune.ai/commander.pho/PhoDibaLongShort2023/e/PHOD-466
basedir: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53
active_data_mode_name: kdiba
Skipping loading from pickled file because force_reload == True.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikes.mat... 

c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\neptune\common\warnings.py:62: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<enum 'SessionBatchProgress'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  warnings.warn(
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\session\Formats\SessionSpecifications.py:140: UserWarning: WARNING: Optional File: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Failure loading .position.npy. Must recompute.



c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\sklearn\manifold\_isomap.py:352: UserWarning: The number of connected components of the neighbors graph is 3 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_ma

Saving updated position results results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy... 2006-6-07_11-26-53.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.interpolated_spike_positions.npy... 2006-6-07_11-26-53.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.laps_info.mat... done.
setting laps object.
session.laps loaded successfully!
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.replay_info.mat... done.
session.replays loaded successfully!
Loading success: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\ripple_df.pkl.
Failure loading .mua.npy. Must recom

...

Applying session filter named "maze2"...
Constraining to epoch with times (start: 1739.1533641185379, end: 1932.4200048116618)
computing neurons mua for session...

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
computing neurons mua for session...



C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to b

using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
due to whitelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (23.923329354140844, 261.8643666456982)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((23.923329354140844, 261.8643666456982), (87.58388755995703, 153.31348420699553))
	 done.
	 spikes_df[time_variable_name]: (10748,) should be less than time_window_edges: (50445,)!
	 spikes_df[time_variable_name]: (10748,) should be less than time_window_edges: (50445,)!


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Analysis\Decoder\reconstruction.py:363: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (50948,)
updating computation_results...
done.
due to whitelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (23.923329354140844, 261.8643666456982)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((23.923329354140844, 261.8643666456982), (87.58388755995703, 153.31348420699553))
	 done.
	 spikes_df[time_variable_name]: (24,) should be less than time_window_edges: (43,)!
	 spikes_df[time_variable_name]: (24,) should be less than time_window_edges: (43,)!
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (41,)
updating computation_results...
done.
due to whitelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (23.923329354140844, 261.8643666456982)
	 d

In [ ]:
# Save to file:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary


In [ ]:
# Get output files:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Build a list of the output files for the good sessions:
session_result_paths = [str(v.joinpath(f'loadedSessPickle.pkl').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]
global_computation_result_paths = [str(v.joinpath(f'output/global_computation_results.pkl').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]

# Write out a GreatlakesOutputs.txt file:
with open('GreatlakesOutputs.txt','w') as f:
    f.write('\n'.join(session_result_paths + global_computation_result_paths))
    # f.write('\n'.join())


In [ ]:
# good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=False, good_only=True)
# good_only_batch_progress_df

[v.get('outputs', None) for v in list(global_batch_run.session_batch_outputs.values()) if v is not None]
# v = list(global_batch_run.session_batch_outputs.values())[0]
# v.get('outputs', None)
  

# [{'local': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl'),
#   'global': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-07_11-26-53/output/global_computation_results.pkl')},
#  {'local': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
#   'global': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl')},
#  ...
# ]
    


# outputs': {'local': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
#    'global': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl')}}

## Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_whitelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_whitelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

# Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]



completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths